# Task 2: Predict Future Stock Prices (Short-Term)

## Objective
Build a machine learning model to predict the next day's closing price of a stock using historical data features (Open, High, Low, Volume).

## Approach
- Use Yahoo Finance data via `yfinance` library
- Apply feature engineering to create lagged variables
- Train Linear Regression and Random Forest models
- Compare model performance using RMSE and visualization

Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stock data fetching
import yfinance as yf

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configure plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

Data Loading and Preprocessing

In [ ]:
# Select stock
ticker = "TSLA"
start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

print(f"Fetching data for {ticker} from {start_date} to {end_date}")

# Download data using yfinance
df = yf.download(ticker, start=start_date, end=end_date, progress=False)

# Display basic info
print(f"\nDataset shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

Data Cleaning and Feature Engineering:

In [ ]:
# Check original columns first
print("Original columns:", df.columns.tolist())
print("Column type:", type(df.columns))

# Flatten column names - handle all yfinance formats
if isinstance(df.columns, pd.MultiIndex):
    # If multi-level columns, take the first level
    df.columns = df.columns.get_level_values(0)
else:
    # If columns have ticker suffix like 'Close TSLA', remove it
    df.columns = [col.split()[0] if ' ' in col else col for col in df.columns]

# Remove any empty columns if exist
df = df.loc[:, ~df.columns.duplicated()]

print("\nFixed columns:", df.columns.tolist())

# Handle any missing values
df = df.dropna()

# Verify 'Close' exists
if 'Close' not in df.columns:
    raise KeyError(f"'Close' column not found. Available columns: {df.columns.tolist()}")

# Create target variable: Next day's closing price
df['Next_Close'] = df['Close'].shift(-1)

# Remove last row (no next day to predict)
df = df[:-1]

print(f"\nDataset shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

Data Visualization and Exploration

In [ ]:
# Plot 1: Historical stock price trend
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Close'], label='Closing Price', linewidth=2)
plt.title(f'{ticker} Stock Price History', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot 2: Feature correlation heatmap
plt.figure(figsize=(10, 8))
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Next_Close']
correlation_matrix = df[features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f')
plt.title('Feature Correlation Matrix', fontsize=14)
plt.tight_layout()
plt.show()

# Plot 3: Distribution of closing prices
plt.figure(figsize=(10, 5))
sns.histplot(df['Close'], kde=True, bins=50)
plt.title('Distribution of Closing Prices')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')
plt.show()

# Plot 4: Volume vs Price relationship
plt.figure(figsize=(10, 6))
plt.scatter(df['Volume'], df['Close'], alpha=0.5)
plt.title('Trading Volume vs Closing Price')
plt.xlabel('Volume')
plt.ylabel('Closing Price ($)')
plt.show()

In [ ]:
# Select features for prediction
feature_columns = ['Open', 'High', 'Low', 'Volume']
target_column = 'Next_Close'

X = df[feature_columns]
y = df[target_column]

# Split data chronologically (important for time series!)
# Use 80% for training, 20% for testing
split_index = int(len(df) * 0.8)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"\nFeature statistics:")
X_train.describe()

Model Training and Evaluation

Model 1: Linear Regression

In [ ]:
# Initialize and train Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
lr_predictions = lr_model.predict(X_test)

# Evaluate Linear Regression
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_predictions))
lr_mae = mean_absolute_error(y_test, lr_predictions)
lr_r2 = r2_score(y_test, lr_predictions)

print("=" * 50)
print("LINEAR REGRESSION RESULTS")
print("=" * 50)
print(f"RMSE: ${lr_rmse:.2f}")
print(f"MAE: ${lr_mae:.2f}")
print(f"R² Score: {lr_r2:.4f}")

# Feature importance (coefficients)
feature_importance_lr = pd.DataFrame({
    'Feature': feature_columns,
    'Coefficient': lr_model.coef_
})
print(f"\nFeature Coefficients:")
print(feature_importance_lr)

Model 2: Random Forest

In [ ]:
# Initialize and train Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    max_depth=10,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_model.predict(X_test)

# Evaluate Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

print("=" * 50)
print("RANDOM FOREST RESULTS")
print("=" * 50)
print(f"RMSE: ${rf_rmse:.2f}")
print(f"MAE: ${rf_mae:.2f}")
print(f"R² Score: {rf_r2:.4f}")

# Feature importance
feature_importance_rf = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)
print(f"\nFeature Importance:")
print(feature_importance_rf)

Plot Actual vs Predicted

In [ ]:
# Create comparison dataframe
test_dates = df.index[split_index:]
comparison_df = pd.DataFrame({
    'Date': test_dates,
    'Actual': y_test.values,
    'Linear_Regression': lr_predictions,
    'Random_Forest': rf_predictions
})
comparison_df.set_index('Date', inplace=True)

# Plot comparison
plt.figure(figsize=(16, 8))
plt.plot(comparison_df.index, comparison_df['Actual'], 
         label='Actual Price', linewidth=2, color='black')
plt.plot(comparison_df.index, comparison_df['Linear_Regression'], 
         label='Linear Regression', linewidth=1.5, linestyle='--', alpha=0.8)
plt.plot(comparison_df.index, comparison_df['Random_Forest'], 
         label='Random Forest', linewidth=1.5, linestyle=':', alpha=0.8)

plt.title(f'{ticker} Stock Price Prediction: Actual vs Predicted', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Zoomed view of last 30 days
plt.figure(figsize=(14, 6))
recent_data = comparison_df.tail(30)
plt.plot(recent_data.index, recent_data['Actual'], 
         label='Actual', marker='o', linewidth=2)
plt.plot(recent_data.index, recent_data['Linear_Regression'], 
         label='Linear Regression', marker='s', linestyle='--')
plt.plot(recent_data.index, recent_data['Random_Forest'], 
         label='Random Forest', marker='^', linestyle=':')
plt.title('Last 30 Days: Detailed Comparison', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Model Comparison and Residual Analysis

In [ ]:
# Model comparison table
comparison_metrics = pd.DataFrame({
    'Metric': ['RMSE', 'MAE', 'R² Score'],
    'Linear Regression': [lr_rmse, lr_mae, lr_r2],
    'Random Forest': [rf_rmse, rf_mae, rf_r2]
})
print("MODEL COMPARISON")
print(comparison_metrics.to_string(index=False))

# Residual plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear Regression residuals
lr_residuals = y_test - lr_predictions
axes[0].scatter(lr_predictions, lr_residuals, alpha=0.5)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('Predicted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Linear Regression: Residual Plot')

# Random Forest residuals
rf_residuals = y_test - rf_predictions
axes[1].scatter(rf_predictions, rf_residuals, alpha=0.5, color='green')
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Predicted Values')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Random Forest: Residual Plot')

plt.tight_layout()
plt.show()

## 9. Explanation of Results and Final Insights

### Results Summary

| Metric | Linear Regression | Random Forest |
|--------|-------------------|---------------|
| **RMSE** | $13.57 | $29.93 |
| **MAE** | $10.70 | $22.56 |
| **R² Score** | 0.961 | 0.810 |

### Key Findings

**1. Linear Regression Outperformed Random Forest**
- Linear Regression achieved **96.1% accuracy** (R² = 0.961), while Random Forest achieved **81.0%** (R² = 0.810)
- Linear Regression's prediction error ($10.70 MAE) was less than half of Random Forest's ($22.56 MAE)
- This indicates **strong linear relationships** between TSLA's OHLC features and next-day closing price

**2. Prediction Accuracy**
- Linear Regression predicts next-day TSLA closing price within **±$10.70 on average**
- Given TSLA's historical price range ($28-$400), this represents approximately **3-5% error margin**
- Random Forest struggled with TSLA's volatility, showing signs of overfitting

**3. Model Suitability**
- **Linear Regression** is better suited for this task due to TSLA's trend-following behavior
- **Random Forest** may capture noise as patterns, leading to higher variance in predictions

### Limitations

- **Short-term only**: Predicts 1 day ahead; accuracy degrades for longer horizons
- **No technical indicators**: Missing RSI, MACD, Moving Averages used by traders
- **No external factors**: Market news, earnings, social sentiment not included
- **Volatility bias**: Extreme price swings may skew model performance

### Future Improvements

- Add technical indicators (SMA, EMA, RSI) for trend analysis
- Include sentiment analysis from news and social media
- Experiment with LSTM/GRU neural networks for time series
- Perform hyperparameter tuning for Random Forest
- Extend prediction horizon to multi-day forecasting

### Conclusion

The **Linear Regression model is the optimal choice** for TSLA short-term price prediction, delivering 96.1% accuracy with an average error of $10.70. However, due to TSLA's inherent volatility and external market influences, this model should serve as a **supportive analytical tool** rather than a standalone trading strategy.